# Jupyter Notebook für Test Cases

In [4]:
%run ../../Setup.ipynb

import unittest
import subprocess
import pymongo
import json
import pickle
import pandas
from bdcc.database.connection.database_connector import DatabaseConnector, get_matches, save_model, load_model
from sklearn.neural_network import MLPClassifier
from api.api_functions import write_match, delete_match, build_dataframe_no_kda, build_dataframe_kda, train_model, predict

In [5]:
class TestGetMatches(unittest.TestCase):
    def test_get_all_matches(self):
        test_list = get_matches()
        test_call = get_matches()
        self.assertListEqual(test_call, test_list)
        self.assertEqual(test_call, test_list)
        self.assertEqual(len(test_call), len(test_list))

    def test_get_one_match_index_0(self):
        test_list = get_matches(num_matches=1)
        test_call = get_matches(num_matches=1)
        self.assertListEqual(test_call, test_list)
        self.assertEqual(test_call, test_list)
        self.assertEqual(len(test_call), 1)
    
    def test_get_one_match_index_10(self):
        test_list = get_matches(start_id=10, num_matches=1)
        test_call = get_matches(start_id=10, num_matches=1)
        self.assertListEqual(test_call, test_list)
        self.assertEqual(test_call, test_list)
        self.assertEqual(len(test_call), 1)
    
    def test_get_10_matches(self):
        test_list = get_matches(num_matches=10)
        test_call = get_matches(num_matches=10)
        self.assertListEqual(test_call, test_list)
        self.assertEqual(test_call, test_list)
        self.assertEqual(len(test_call), 10)
    
    def test_get_matches_return_type(self):
        test_call = get_matches(num_matches=1)
        self.assertIsInstance(test_call, list)
        self.assertIsInstance(test_call[0], dict)

In [6]:
class TestDatabaseConnector(unittest.TestCase):
    def setUp(self):
        self.db_con = DatabaseConnector()
    
    def test_DataBaseConnector_init(self):
        db_con_temp = DatabaseConnector()
        urls = ['localhost', 'mongodb-ki']
        # url
        self.assertTrue(db_con_temp.url in urls)
        db_con_temp.url = '127.0.0.1'
        self.assertEqual(db_con_temp.url, '127.0.0.1')
        # database
        self.assertEqual(db_con_temp.database, 'Data-KI')
        db_con_temp.database = 'Data'
        self.assertEqual(db_con_temp.database, 'Data')
        # collection
        self.assertEqual(db_con_temp.collection, 'MatchesDto')
        db_con_temp.collection = 'Matches'
        self.assertEqual(db_con_temp.collection, 'Matches')

    def test_DataBaseConnector_connect(self):
        test_call = self.db_con.connect()
        self.assertIsInstance(test_call, type(self.db_con))
        self.assertIsInstance(test_call.client, pymongo.MongoClient)
        self.assertIsInstance(test_call.db, pymongo.database.Database)
        self.assertIsInstance(test_call.connection, type(self.db_con))
        self.assertIsInstance(test_call.active_collection, pymongo.collection.Collection)
    
    def test_DataBaseConnector_disconnect(self):
        # client nicht verbunden
        self.assertRaises(ConnectionError, lambda: self.db_con.disconnect())
        test_call = self.db_con.connect()
        self.assertIsInstance(test_call, type(self.db_con))
    
    def test_DataBaseConnector_getCollectionNames(self):
        self.db_con.connect()
        test_call = self.db_con.getCollectionNames()
        self.assertIsInstance(test_call, list)

    def test_DataBaseConnector_getCollection(self):
        self.db_con.connect()
        test_call = self.db_con.getCollection('MatchesDto')
        self.assertIsInstance(test_call, pymongo.collection.Collection)

    def test_DataBaseConnector_create(self):
        self.db_con.connect()
        match = json.loads("{\"match_id\": 123}")
        self.assertTrue(self.db_con.create(match).acknowledged)
        loaded_match = self.db_con.get(id=123)[0]
        self.assertEqual(match['match_id'], loaded_match['match_id'])
        self.db_con.disconnect()
        delete_match(123)
    
    def test_DataBaseConnector_get(self):
        match = json.loads("{\"match_id\": 123}")
        write_match(match)
        self.db_con.connect()
        test_call_all = self.db_con.get()
        test_call_one = self.db_con.get(id=123)
        self.assertIsInstance(test_call_all, pymongo.cursor.Cursor)
        self.assertGreaterEqual(len(list(test_call_all)), 1)
        self.assertIsInstance(test_call_one, pymongo.cursor.Cursor)
        self.assertEqual(len(list(test_call_one)), 1)
        delete_match(123)
    
    def test_DataBaseConnector_update(self):
        self.db_con.connect()
        match = json.loads("{\"match_id\": 123, \"radiant_win\": true}")
        new_match = json.loads("{\"match_id\": 123, \"radiant_win\": false}")
        write_match(match)
        self.db_con.update(123, new_match)
        updated_match = self.db_con.get(123)[0]
        self.assertEqual(updated_match['radiant_win'], new_match['radiant_win'])
        delete_match(123)

    def test_DataBaseConnector_remove(self):
        self.db_con.connect()
        match = json.loads("{\"match_id\": 123, \"radiant_win\": true}")
        write_match(match)
        self.db_con.remove(123)
        self.assertRaises(IndexError, lambda: self.db_con.get(123)[0])

    def test_DataBaseConnector_save_model(self):
        db_con = DatabaseConnector(collection="Models")
        db_con.connect()
        model_name = "kda"
        model = MLPClassifier() # define model
        pickled_model = pickle.dumps(model) 
        self.assertTrue(db_con.save_model(pickled_model, model_name).acknowledged)
        # load model from database
        models = list(db_con.get())
        db_con.disconnect()
        # load models
        loaded_model = [pickle.loads(model['model']) for model in models if model['name'] == model_name][0]
        self.assertEqual(type(model), type(loaded_model))

    def test_DataBaseConnector_update_model(self):
        db_con = DatabaseConnector(collection="Models")
        db_con.connect()
        model_name = "kda"
        model = MLPClassifier() # define model
        pickled_model = pickle.dumps(model) 
        db_con.save_model(pickled_model, model_name)
        # define updated model
        updated_model = MLPClassifier(activation='tanh')
        updated_pickled_model = pickle.dumps(updated_model) 
        db_con.update_model(updated_pickled_model, model_name)
        # load model from database
        models = list(db_con.get())
        db_con.disconnect()
        # load models
        loaded_model = [pickle.loads(model['model']) for model in models if model['name'] == model_name][0]
        self.assertEqual(type(updated_model), type(loaded_model))
        self.assertEqual(updated_model.get_params(), loaded_model.get_params())
        

In [7]:
class TestSaveModel(unittest.TestCase):
    def setUp(self):
        model_name_kda = "kda"
        model_name_no_kda = "no_kda"
        self.model = MLPClassifier() # define model
        save_model(self.model, model_name_kda)
        save_model(self.model, model_name_no_kda)
        # load model from database
        db_con = DatabaseConnector(collection="Models")
        db_con.connect()
        models = list(db_con.get())
        db_con.disconnect()
        # load models
        self.loaded_model_kda = [pickle.loads(model['model']) for model in models if model['name'] == model_name_kda][0]
        self.loaded_model_no_kda = [pickle.loads(model['model']) for model in models if model['name'] == model_name_no_kda][0]
    
    def test_save_model_kda(self):
        self.assertEqual(type(self.model), type(self.loaded_model_kda))
        self.assertEqual(self.model.get_params(), self.loaded_model_kda.get_params())

    def test_save_model_no_kda(self):
        self.assertEqual(type(self.model), type(self.loaded_model_no_kda))
        self.assertEqual(self.model.get_params(), self.loaded_model_no_kda.get_params())

    def test_save_model_invalid(self):
        model_name = "invalid_name"
        self.assertRaises(NameError, lambda: save_model(self.model, model_name))

In [8]:
class TestLoadModel(unittest.TestCase):
    def setUp(self):
        self.model_name_kda = "kda"
        self.model_name_no_kda = "no_kda"
        self.model = MLPClassifier() # define model
        save_model(self.model, self.model_name_kda)
        save_model(self.model, self.model_name_no_kda)

    def test_load_model_kda(self):
        loaded_model_kda = load_model(self.model_name_kda)
        self.assertEqual(type(self.model), type(loaded_model_kda))
        self.assertEqual(self.model.get_params(), loaded_model_kda.get_params())

    def test_load_model_no_kda(self):
        loaded_model_no_kda = load_model(self.model_name_no_kda)
        self.assertEqual(type(self.model), type(loaded_model_no_kda))
        self.assertEqual(self.model.get_params(), loaded_model_no_kda.get_params())

    def test_load_model_fail(self):
        model_name = "invalid_name"
        self.assertRaises(NameError, lambda: load(self.model, model_name))

In [9]:
class TestWriteMatch(unittest.TestCase):
    def setUp(self):
        self.match_id = 123
        self.match = json.loads("{\"match_id\": 123}")
        self.http_status_created = 201
        self.http_status_conflicted = 409

    def test_write_match_success(self):
        self.assertEqual(write_match(self.match), self.http_status_created)
        delete_match(self.match_id)

    def test_write_match_duplicate(self):
        http_status = write_match(self.match)
        self.assertEqual(http_status, self.http_status_created)
        self.assertEqual(write_match(self.match), self.http_status_conflicted)
        delete_match(self.match_id)

In [10]:
class TestDeleteMatch(unittest.TestCase):
    def setUp(self):
        self.match_id = 123
        self.match = json.loads("{\"match_id\": 123}")
        self.http_status_no_content = 204
        self.http_status_reset_content = 205
        write_match(self.match)

    def test_delete_match_success(self):
        self.assertEqual(delete_match(self.match_id), self.http_status_reset_content)
    def test_delete_match_no_existing_match(self):
        invalid_id = 0
        self.assertEqual(delete_match(invalid_id), self.http_status_no_content)
        

In [11]:
class TestBuildDataFrame(unittest.TestCase):
    def setUp(self):
        self.dataframe = [{'match_id': 6049289816, 'radiant_win': True, 'players': [{'pings': 2, 'assists': 3, 'deaths': 1, 'kills': 21, 'win': True}, {'pings': 2, 'assists': 3, 'deaths': 1, 'kills': 21, 'win': True}, {'pings': 2, 'assists': 3, 'deaths': 1, 'kills': 21, 'win': True}, {'pings': 2, 'assists': 3, 'deaths': 1, 'kills': 21, 'win': True}, {'pings': 2, 'assists': 3, 'deaths': 1, 'kills': 21, 'win': True}, {'pings': 2, 'assists': 3, 'deaths': 1, 'kills': 21, 'win': False}, {'pings': 2, 'assists': 3, 'deaths': 1, 'kills': 21, 'win': False}, {'pings': 2, 'assists': 3, 'deaths': 1, 'kills': 21, 'win': False}, {'pings': 2, 'assists': 3, 'deaths': 1, 'kills': 21, 'win': False}, {'pings': 2, 'assists': 3, 'deaths': 1, 'kills': 21, 'win': False}]}]
        self.dataframe_no_kda_check = pandas.DataFrame([[10, True], [10, False]], columns =['ttlPings', 'win'])
        self.dataframe_kda_check = pandas.DataFrame([[2, 3, 1, 21, True], [2, 3, 1, 21, True], [2, 3, 1, 21, True] , [2, 3, 1, 21, True], [2, 3, 1, 21, True], [2, 3, 1, 21, False], [2, 3, 1, 21, False], [2, 3, 1, 21, False], [2, 3, 1, 21, False], [2, 3, 1, 21, False]], columns =['pings', 'assists', 'deaths', 'kills', 'win'])
        
    def test_build_dataframe_no_kda_type(self):
        self.assertIsInstance(build_dataframe_no_kda(self.dataframe), pandas.core.frame.DataFrame)
        
    def test_build_dataframe_no_kda_as_expected(self):
        self.assertEqual(self.dataframe_no_kda_check.equals(build_dataframe_no_kda(self.dataframe)), True)
        
    def test_buld_dataframe_kda_type(self):
        self.assertIsInstance(build_dataframe_kda(self.dataframe), pandas.core.frame.DataFrame)
    
    def test_build_dataframe_kda_as_expected(self):
        self.assertEqual(self.dataframe_kda_check.equals(build_dataframe_kda(self.dataframe)), True)

In [12]:
class TestTrainModel(unittest.TestCase):
    def setUp(self):
        self.http_status_ok = 200
        self.http_status_badRequest = 400
        self.model_name_kda = "kda"
        self.model_name_no_kda = "no_kda"
        
    def test_train_model_success(self):
        self.assertEqual(train_model(self.model_name_kda), self.http_status_ok)
        self.assertEqual(train_model(self.model_name_no_kda), self.http_status_ok)
    
    def test_train_model_wrong_name(self):
        self.assertEqual(train_model("123"), self.http_status_badRequest)

In [13]:
class TestPredict(unittest.TestCase):
    def setUp(self):
        self.http_status_ok = 200
        self.http_status_badRequest = 400
        self.model_name_kda = "kda"
        self.model_name_no_kda = "no_kda"
        self.dataframe = [{'match_id': 6049289816, 'radiant_win': True, 'players': [{'pings': 2, 'assists': 3, 'deaths': 1, 'kills': 21, 'win': True}, {'pings': 2, 'assists': 3, 'deaths': 1, 'kills': 21, 'win': True}, {'pings': 2, 'assists': 3, 'deaths': 1, 'kills': 21, 'win': True}, {'pings': 2, 'assists': 3, 'deaths': 1, 'kills': 21, 'win': True}, {'pings': 2, 'assists': 3, 'deaths': 1, 'kills': 21, 'win': True}, {'pings': 2, 'assists': 3, 'deaths': 1, 'kills': 21, 'win': False}, {'pings': 2, 'assists': 3, 'deaths': 1, 'kills': 21, 'win': False}, {'pings': 2, 'assists': 3, 'deaths': 1, 'kills': 21, 'win': False}, {'pings': 2, 'assists': 3, 'deaths': 1, 'kills': 21, 'win': False}, {'pings': 2, 'assists': 3, 'deaths': 1, 'kills': 21, 'win': False}]}]
        
    def test_predict_model_name(self):
        self.assertEqual(predict(self.model_name_kda, self.dataframe)[0], self.http_status_ok)
        self.assertEqual(predict(self.model_name_no_kda, self.dataframe)[0], self.http_status_ok)
        
    def test_predict_model_name_wrong(self):
        self.assertEqual(predict("123", self.dataframe)[0], self.http_status_badRequest)
        
    def test_predict_matches_type(self):
        self.assertIsInstance(predict(self.model_name_no_kda, self.dataframe)[1], str)
        self.assertIsInstance(predict(self.model_name_kda, self.dataframe)[1], str)

In [14]:
def suite():
    """Funktion zum Erstellen einer Test Suite für die Ausführung der Unittests."""
    suite = unittest.TestSuite()
    suite.addTest(unittest.makeSuite(TestGetMatches, 'test'))
    suite.addTest(unittest.makeSuite(TestDatabaseConnector, 'test'))
    suite.addTest(unittest.makeSuite(TestSaveModel, 'test'))
    suite.addTest(unittest.makeSuite(TestLoadModel, 'test'))
    suite.addTest(unittest.makeSuite(TestWriteMatch, 'test'))
    suite.addTest(unittest.makeSuite(TestDeleteMatch, 'test'))
    suite.addTest(unittest.makeSuite(TestBuildDataFrame, 'test'))
    suite.addTest(unittest.makeSuite(TestTrainModel, 'test'))
    suite.addTest(unittest.makeSuite(TestPredict, 'test'))
    return suite

In [15]:
def execute_unittests():
    """Funktion zum Ausführen der Unittests."""
    unittest.main(argv=[''], verbosity=2, exit=False, defaultTest='suite')